In [1]:
!pip --quiet install --upgrade torch torchaudio jiwer datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

## PARTE 1: IMPORT e CARICAMENTO del dataset LibriSpeech

In [2]:
import torchaudio.transforms as T
from torchaudio.transforms import MelSpectrogram, FrequencyMasking, TimeMasking, AmplitudeToDB
from torchaudio.datasets import LIBRISPEECH
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn import CTCLoss
from torch.utils.data import DataLoader, random_split, Subset
from jiwer import wer, cer
import time
import random
import re
import os
from pprint import pprint
from torch.amp import autocast, GradScaler
from torch import amp
import torch.nn.functional as F
from tqdm.auto import tqdm
import math

In [3]:
data_dir = "./data"

train_dataset = LIBRISPEECH(root=data_dir, url="train-clean-100", download=True)
dev_dataset = LIBRISPEECH(root=data_dir, url="dev-clean", download=True)
test_dataset = LIBRISPEECH(root=data_dir, url="test-clean", download=True)
print("Caricamento dataset LibriSpeech avvenuto!")

100%|██████████| 5.95G/5.95G [07:28<00:00, 14.2MB/s]
100%|██████████| 322M/322M [00:24<00:00, 14.0MB/s]
100%|██████████| 331M/331M [00:26<00:00, 12.9MB/s]


Caricamento dataset LibriSpeech avvenuto!


In [4]:
if torch.cuda.is_available():
    print(f"GPU disponibile: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print("GPU non disponibile, usando CPU")

print(f"Device utilizzato: {device}")

GPU disponibile: NVIDIA L4
Memoria GPU: 22.2 GB
Device utilizzato: cuda


## PARTE 2: VOCABOLARIO

In [5]:
def normalize_text(text):
    text = text.upper()
    text = re.sub(r"[^A-Z ]+", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def build_vocab_dict(dataset):
    normalized_texts = [normalize_text(example[2]) for example in dataset]

    all_text = " ".join(normalized_texts)
    unique_chars = sorted(set(all_text) - {" "})
    unique_chars.append("|")

    vocab_dict = {c: i for i, c in enumerate(unique_chars)}

    vocab_dict["<blank>"] = len(vocab_dict)
    vocab_dict["<pad>"] = len(vocab_dict)
    vocab_dict["<unk>"] = len(vocab_dict)

    return vocab_dict

vocab_dict = build_vocab_dict(train_dataset)
id2char = {i: c for c, i in vocab_dict.items()}
blank_id = vocab_dict["<blank>"]

def text_to_int(text):
    text = normalize_text(text).replace(" ", "|")
    return [vocab_dict.get(c, vocab_dict["<unk>"]) for c in text]

## PARTE 3: MODELLO


In [17]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-torch.log(torch.tensor(10000.0)) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return x

class SimpleASRModel(nn.Module):
    def __init__(self, vocab_size):
        super(SimpleASRModel, self).__init__()

        # 1. CNN per feature extraction (2 layer)
        self.cnn = nn.Sequential(
            nn.Conv1d(80, 256, kernel_size=7, stride=2, padding=5),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout1d(0.2)
        )

        # 2. Positional Encoding
        self.positional_encoding = PositionalEncoding(d_model=256)

        # 3. Transformer Encoder
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=256,
                nhead=8,
                dim_feedforward=512,
                dropout=0.1
            ),
            num_layers=3
        )

        self.classifier = nn.Linear(256, vocab_size)

    def forward(self, x):
        x = self.cnn(x)
        x = x.permute(2,0,1)
        x = self.positional_encoding(x)
        x = self.transformer(x)
        x = self.classifier(x)

        return x

## PARTE 4: COLLATE

In [18]:
mel_transform = MelSpectrogram(sample_rate=16000, n_mels=80)
freq_mask = FrequencyMasking(freq_mask_param=30)
time_mask = TimeMasking(time_mask_param=100)
db_transform = AmplitudeToDB()

def collate(batch):
  spectrograms, labels, input_lengths, label_lengths = [], [], [], []

  for waveform, _, transcript, *_ in batch:
    spec = mel_transform(waveform).squeeze(0).transpose(0, 1)
    spec = db_transform(spec)

    mean = spec.mean()
    std  = spec.std()
    spec = (spec - mean) / (std + 1e-5)

    if torch.is_grad_enabled() and random.random() < 0.5:
      freq_mask.mask_param = random.randint(10, 30)
      time_mask.mask_param = random.randint(50, 100)
      spec = freq_mask(spec)
      spec = time_mask(spec)
    spectrograms.append(spec)

    label = torch.tensor(text_to_int(transcript))
    labels.append(label)

    input_lengths.append(spec.size(0) // 2)
    label_lengths.append(len(label))

  spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).transpose(1, 2)
  labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)
  return spectrograms, labels, torch.tensor(input_lengths), torch.tensor(label_lengths)

## PARTE 5: TRAIN LOOP

In [21]:
def split_dataset(dataset, portion_size, train_ratio=0.8, shuffle=True, seed=None):
    total = len(dataset)
    n = min(portion_size, total)

    g = torch.Generator()
    if seed is not None:
        g.manual_seed(seed)

    if shuffle:
        perm = torch.randperm(total, generator=g)[:n].tolist()
    else:
        perm = list(range(n))

    base = Subset(dataset, perm)

    n_train = int(train_ratio * n)
    n_dev   = n - n_train
    if shuffle:
        train_subset, dev_subset = random_split(
            base, [n_train, n_dev], generator=g
        )
    else:
        train_idx = perm[:n_train]
        dev_idx   = perm[n_train:]
        train_subset = Subset(dataset, train_idx)
        dev_subset   = Subset(dataset, dev_idx)

    return train_subset, dev_subset

In [20]:
torch.backends.cudnn.benchmark = True

model = SimpleASRModel(vocab_size=len(vocab_dict)).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
criterion = CTCLoss(blank=blank_id, zero_infinity=True).to(device)
scaler = GradScaler(device="cuda")

train_batch_size = 32
eval_batch_size = 32

small_train, small_dev = split_dataset(train_dataset, portion_size=10000, train_ratio=0.7, shuffle=True, seed=42)

train_loader = DataLoader(
    small_train,
    batch_size=train_batch_size,
    shuffle=True,
    collate_fn=collate,
    pin_memory=True,
    num_workers=4,
    persistent_workers=True,
)
dev_loader = DataLoader(
    small_dev,
    batch_size=eval_batch_size,
    shuffle=False,
    collate_fn=collate,
    pin_memory=True,
    num_workers=4,
    persistent_workers=True,
)

print("Numero di batch per epoca:", len(train_loader))

def ctc_decode(log_probs, blank_id, id2char, blank_thresh=None):

    preds = log_probs.permute(1, 0, 2).cpu()
    preds = preds.argmax(dim=-1)

    if blank_thresh is not None:
        blank_p = log_probs.exp().permute(1, 0, 2)[..., blank_id].cpu()
        preds[blank_p > blank_thresh] = -1

    decoded = []
    for seq in preds:
        uniq = torch.unique_consecutive(seq, dim=0)
        toks = uniq[uniq >= 0].tolist()
        text = ''.join(id2char[t] for t in toks)
        decoded.append(text.replace('|', ' '))
    return decoded

def monitor_gpu():
  if torch.cuda.is_available():
      allocated = torch.cuda.memory_allocated() / 1024**3
      reserved = torch.cuda.memory_reserved() / 1024**3
      print(f"GPU Memory - Allocated: {allocated:.2f}GB, Reserved: {reserved:.2f}GB")

patience = 10
best_val_loss = float('inf')
no_improve_epochs = 0
monitor_gpu()

for epoch in range(1, 101):

    # ======= TRAINING ========
    model.train()
    total_train_loss = 0.0
    start_time = time.time()

    for specs, labels, input_lengths, label_lengths in tqdm(train_loader, desc=f"Epoch {epoch}"):
        specs, labels = specs.to(device), labels.to(device)

        optimizer.zero_grad()

        with autocast("cuda"):
          outputs = model(specs)
          log_probs = F.log_softmax(outputs, dim=2)
          train_loss = criterion(log_probs, labels, input_lengths, label_lengths)

        scaler.scale(train_loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_train_loss += train_loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    duration = time.time() - start_time

    # ======= VALUTAZIONE ========

    model.eval()
    total_val_loss = 0
    all_preds, all_targets = [], []

    with torch.no_grad():
      for specs, labels, input_lengths, label_lengths in tqdm(dev_loader, desc="Validation"):
          specs, labels = specs.to(device), labels.to(device)

          with autocast("cuda"):
            outputs = model(specs)
            log_probs = F.log_softmax(outputs, dim=2)
            val_loss = criterion(log_probs, labels, input_lengths, label_lengths)

          total_val_loss += val_loss.item()

          if epoch % 5 == 0:
                pred_texts = ctc_decode(log_probs, blank_id, id2char, blank_thresh=0.4)

                for lab, L in zip(labels.cpu(), label_lengths):
                    target_ids = lab[:L].tolist()
                    target_text = "".join([id2char[idx] for idx in target_ids])
                    target_text = target_text.replace("|", " ")
                    all_targets.append(target_text)

                all_preds.extend(pred_texts)

    avg_val_loss = total_val_loss / len(dev_loader)

    print(f"EPOCA: {epoch} | TRAIN_LOSS: {avg_train_loss:.4f} | VAL_LOSS: {avg_val_loss:.4f} | DURATA: {duration:.2f}s")

    if epoch % 5 == 0 and all_preds:
        wer_score = wer(all_targets, all_preds)
        cer_score = cer(all_targets, all_preds)
        print(f"WER: {wer_score:.4f}, CER: {cer_score:.4f}")

        print("\nEsempi di predizioni:")
        for i in range(min(3, len(all_preds))):
            print(f"Ref:  {all_targets[i]}")
            print(f"Pred: {all_preds[i]}")

    # ======= EARLY STOPPING ========

    scheduler.step(avg_val_loss)
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        no_improve_epochs = 0
        torch.save(model.state_dict(), 'best_asr.pth')
        print("Nuovo best model salvato")
        print("-" * 50)
    else:
        no_improve_epochs += 1
        print(f"Nessun miglioramento ({no_improve_epochs}/{patience})")
        print("-" * 50)
        if no_improve_epochs >= patience:
            print("Early stopping")
            break

print("Training completato!")


Numero di batch per epoca: 219
GPU Memory - Allocated: 1.16GB, Reserved: 2.12GB


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1:   0%|          | 0/219 [00:00<?, ?it/s]

Validation:   0%|          | 0/94 [00:00<?, ?it/s]

EPOCA: 1 | TRAIN_LOSS: 2.9019 | VAL_LOSS: 2.4654 | DURATA: 19.17s
Nuovo best model salvato
--------------------------------------------------


...

Validation:   0%|          | 0/94 [00:00<?, ?it/s]

EPOCA: 99 | TRAIN_LOSS: 1.2861 | VAL_LOSS: 1.0911 | DURATA: 18.42s
→ Nessun miglioramento (2/10)
--------------------------------------------------


Epoch 100:   0%|          | 0/219 [00:00<?, ?it/s]

Validation:   0%|          | 0/94 [00:00<?, ?it/s]

EPOCA: 100 | TRAIN_LOSS: 1.2816 | VAL_LOSS: 1.0919 | DURATA: 18.65s
WER: 0.8379, CER: 0.4156

Esempi di predizioni:
Ref:  I NEVER SAID MISSUS CAREY NO I NEVER HEARD A THING LIKE THAT WHAT NOT WHEN YOU CUT UP THE JACKASS AND SOLD IT FOR VEAL CUTLETS MOTHER HOLD YOUR TONGUE MISTER IMPERENCE SAID THE WIDOW ITS VERY WELL KNOWN YOURE NO CHRISTIAN AND WHOLL BELIEVE WHAT YOU SAY
Pred: IINENEMVERSAID MI ES CARY NO IN NEVER<blank>TERDTISING LIGAT AT NOCKWHENYOU CUT OTI GAC<blank>CAS<blank>MSOLDE RO VEO CUTLUTS MO<blank> HER HOLD YOUR TONG MISTR EMPERIN SHAID THE WI DO ICH VERY WELL NO Y UKNO CREISTION ANWOLBLLY WECUSOY
Ref:  WOULD IT BE TO PROMOTE HIS HAPPINESS TO ENABLE HIM TO MARRY SUCH A GIRL SHE MUST BE AN UNPRINCIPLED ONE OR SHE COULD NOT HAVE USED YOUR BROTHER SO AND HOW STRANGE AN INFATUATION ON FREDERICKS SIDE A GIRL
Pred: WHATY BE TO PROMOT HIS APENINS TO LIN A<blank>BE<blank>LIN TOMARYSUTCE GERF SHEM MUSD BEAN ON PRINS IPLD WOM ARSHE CRO NOT OFE U TO RETHERES SO AND HOS TRANGININ FATU